In [14]:
import pandas as pd
import geopandas as gpd
from shapely import Point

This notebooks loads the Counterglow dataset, does some data cleaning, and exports it as a geojson to be used in mapping.

In [2]:
df = pd.read_csv("../data/Counterglow+Facility+List+Complete.csv")

In [3]:
df.head()

,Name,Lat,Lat.1,Address,City,State,County,Description,Business/company name,Postal address,Phone number,Region,Facility name,Number of animals,Full address,Website URL,Postcode,Suburb/city,Contracted to,Farm Type
0,78,43.396729,-95.923149,"1757 Lily Avenue George 51237, United States",NaN,IA,Lyon County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
1,84,42.925350,-96.429291,"1804 500th Street Hawarden 51023, United States",NaN,IA,Sioux County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
2,87,40.658218,-92.410202,"21166 Mahogany Avenue Bloomfield 52537, United...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
3,88,40.643219,-92.409889,"27268 Mahogany Avenue Bloomfield 52537, United...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
4,89,40.660851,-92.421219,"21166 280th Street Bloomfield 52537, United St...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)


In [4]:
len(df)

29853

In [20]:
df["Farm Type"].unique()

array(['Pigs (Meat)', 'Cows (Meat)',
       'Cows, Bulls, Horses, Other Animals - Rodeos', 'Unknown',
       'Cows (Dairy, Meat)', nan,
       'Mink/Foxes/Rabbits/Other Mammals (Fur)', 'Slaughterhouses',
       'Chickens (Meat)', 'Chickens (Eggs)',
       'Fish & Marine Animals (Meat)', 'Goats (Meat)', 'Turkeys (Meat)',
       'Live Animal Saleyards', 'Alligators & Crocodiles (Skin, Meat)',
       'Sheep (Meat)', 'Horse Racetracks', 'Scientific Testing Labs',
       'Miscellaneous', 'Corporate Office/Hq',
       'Chickens & Other Birds (Meat)', 'Honey Bees (Honey)',
       'Live Animal Export', 'Rabbits (Fur, Meat)', 'Zoos',
       'Ducks (Meat)', 'Horses',
       'Rendering (Processing leftover animal body parts from slaughterhouses)',
       'Pullets - Birds (Meat)', 'Pigs (Meat) & Cows (Dairy, Meat)',
       'Greyhound Racetracks', 'Quails (Meat)',
       'Multiple Types of Animals', 'Koalas',
       'Deer (Hunting and/or Butchering)', 'Circuses',
       'Pigs & Cows (Meat)', 'Mice 

In [21]:
df_poultry = df[(df["Farm Type"] == "Chickens (Meat)") | (df["Farm Type"] == "Chickens & Other Birds (Meat)")]

In [22]:
len(df_poultry)

2735

In [27]:
len(df_poultry[df_poultry.State == "AR"])

22

In [40]:
df_poultry.State.unique()

array(['IA', 'AL', 'NE', 'MS', 'TX', 'IN', 'IL', 'NC', 'MD', 'MO', 'PA',
       'OH', 'WI', 'ID', 'GA', 'OK', 'WA', 'LA', 'VA', nan, 'SD', 'CA',
       'AR', 'SC', 'MI', 'KY', 'DE', 'OR', 'TN', 'MN', 'KS', 'NJ', 'MT',
       'NY', 'ME', 'VT', 'ND', 'WV', 'CO'], dtype=object)

In [9]:
df_states = gpd.read_file("../data/all_states.geojson")

In [28]:
df_states[df_states.state == "Arkansas"]

,state,plant_access,area,geometry
4,Arkansas,1,4.062816e+10,"MULTIPOLYGON (((-94.00708 33.20157, -94.01254 ..."
5,Arkansas,2,2.535141e+10,"MULTIPOLYGON (((-94.40061 34.34164, -94.44440 ..."
6,Arkansas,3,2.082532e+10,"MULTIPOLYGON (((-90.74369 35.56358, -90.76770 ..."
7,Arkansas,4,1.228056e+10,"MULTIPOLYGON (((-94.33686 35.57354, -94.33547 ..."


In [38]:
abb2state = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}


In [ ]:
# geojson has full state names and Counterglow data has state abbreviations
# convert from abbreviation to actual state name
list_of_farms = []
for state in df_poultry.State.unique():
    for i, farm in df_poultry[df_poultry.State == state].iterrows():
        pt = Point(farm["Lat.1"], farm["Lat"])
        for k, area in df_states[df_states.state == abb2state[state]].iterrows():
            if area.geometry.contains(pt):
                farm_data = {
                    "state": abb2state[state],
                    "geometry": pt,
                    "address": farm.Address,
                    "company": farm["Business/company name"],
                    "plant_access": area.plant_access
                }
                # print(farm_data)
                list_of_farms.append(farm_data)

In [54]:
area

state                                               West Virginia
plant_access                                                    4
area                                             726692813.153427
geometry        MULTIPOLYGON (((-79.05848296706 38.88617476916...
Name: 77, dtype: object

In [56]:
contained_poultry_farms = gpd.GeoDataFrame(list_of_farms)

In [57]:
contained_poultry_farms.head()

,state,geometry,address,company,plant_access
0,Iowa,POINT (-92.27755 43.46849),"3306 Oak Avenue Lime Springs 52155, United States",NaN,1
1,Iowa,POINT (-91.57521 43.03394),"13242 Aspen Avenue Postville 52162, United States",NaN,1
2,Iowa,POINT (-91.55061 42.98755),"12673 160th Street Postville 52162, United States",NaN,1
3,Iowa,POINT (-91.81220 43.12277),"1286 210th Avenue Ossian 52161, United States",NaN,1
4,Iowa,POINT (-91.59761 43.07805),"10130 U.S. 18 Postville 52162, United States",NaN,1


In [59]:
# export counterglow data to geojson
contained_poultry_farms.to_file("../data/counterglow_geojson.geojson", driver="GeoJSON")